In [1]:
import os
import ee
import geemap
ee.Initialize()
Map = geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

In [2]:
AOI = ee.Geometry.Polygon([[[-7.590167, 31.968456],
          [-7.590167, 32.219754],
          [-7.114261, 32.219754],
          [-7.114261, 31.968456]]])

In [3]:
L8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA') \
.filterDate('2020-08-1','2020-08-30') \
.filterBounds(AOI) \
.filter(ee.Filter.lt('CLOUD_COVER', 10))




In [4]:
img = ee.Image(L8.median()).clip(AOI)
viz = {'min':0, 'max':0.4, 'bands':"B4,B3,B2"}


In [5]:
NSI = img.expression(
    '((SWIR1 - SWIR2) / (SWIR1 - NIR))', {
      'NIR' : img.select('B5'),
      'SWIR1': img.select('B6'),
      'SWIR2': img.select('B7')
}).rename('NSI')
nsiParams = {'min': 0, 'max': 4, 'palette': ['yellow', 'orange','grey', 'white']}

SSSI_1 = img.expression(
    '(SWIR1 - SWIR2)', {
      'SWIR1': img.select('B6'),
      'SWIR2': img.select('B7')
}).rename('SSSI_1')

sssi_1_Params = {'min': 0.04, 'max': 0.19, 'palette': ['yellow', 'orange','grey', 'white']}

SSSI_2 = img.expression(
    '((SWIR1 * SWIR2) - (SWIR2 * SWIR2)) / SWIR1', {
      'SWIR1': img.select('B6'),
      'SWIR2': img.select('B7')
}).rename('SSSI_2')

sssi_2_Params = {'min': 0.04, 'max': 0.14, 'palette': ['yellow', 'orange','grey', 'white']}

In [20]:
legend_keys = ['High', '-', '-', '-']
 #colorS can be defined using either hex code or RGB (0-255, 0-255, 0-255)
legend_colors = ['#FFFF00', '#FFA07A', '#808080', '#FFFFFF']
# legend_colors = [(255, 0, 0), (127, 255, 0), (127, 18, 25), (36, 70, 180), (96, 68 123)]

Map.add_legend(legend_keys=legend_keys, legend_colors=legend_colors, position='bottomleft')


In [6]:
Map.addLayer(img,viz,"Landsat 8")
Map.addLayer(NSI,nsiParams,"NSI")
Map.addLayer(SSSI_1, sssi_1_Params, 'SSSI_1')
Map.addLayer(SSSI_2, sssi_2_Params, 'SSSI_2')